In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys
import os
import numpy as np
from tqdm import tqdm
import glob

import torchaudio
from speechbrain.pretrained.interfaces import foreign_class

torchvision is not available - cannot save figures


In [16]:
classifier = foreign_class(source="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier", savedir="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", run_opts={"device":"cuda"})

speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


In [3]:
dataset_dir = "/mntcephfs/lee_dataset/tts/LibriTTS_R/"
feat_base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/LibriTTS_R/features/"
fs = 24000

speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*/*")]
speakers.sort()

In [27]:
save = True

files = []
for spk in tqdm(speakers):
    files += glob.glob(dataset_dir + f"*/{spk}/*/*.wav")
files.sort()
for path in tqdm(files):
    signal, org_sr = torchaudio.load(path)
    signal = torchaudio.functional.resample(signal, orig_freq=org_sr, new_freq=fs)
    embeddings =  classifier.encode_batch(signal)[0].detach().cpu().numpy()
    
    savepath = feat_base_dir + "/".join(path.split("/")[-4:])[:-4] + "_accentembedding.npy"
    if save:
        np.save(savepath, embeddings)

100%|██████████| 160267/160267 [2:20:08<00:00, 19.06it/s]   
